# 10 Fold CV

In [6]:
import pandas as pd
from Bio import SeqIO
import os
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

import imblearn
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc
import os 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
os.chdir("/home/t326h379/OGP")


df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

In [3]:
a = random.sample(range(1, 1000000), 1)

In [8]:
for i in a:
    seed = i
    print("Seed : ",seed)
    rus = RandomUnderSampler(random_state = seed)
    X_train, y_train = rus.fit_resample(X_train_full,y_train_full)

    X_train, y_train = shuffle(X_train,y_train)

    print(X_train.shape)
    print(y_train.shape)

    MCC = []
    kfold = StratifiedKFold(n_splits=10)
    cvscores, auc_scores, sn, sp, acc = list(), list(), list(), list(), list()

    for train_index, test_index in kfold.split(X_train, y_train):
        xtrain, xval = X_train[train_index], X_train[test_index]
        ytrain, yval = y_train[train_index], y_train[test_index]      

        X_train_10= xtrain
        Y_train_10=ytrain

        from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc
        from imblearn.under_sampling import RandomUnderSampler

        rus = RandomUnderSampler(random_state = seed)

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train_10, Y_train_10)
       
        
        y_pred = np.array(clf.predict(xval))
   
        confusion = confusion_matrix(yval,y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval,y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval,y_pred))
        print("Accuracy on test set:   ",accuracy_score(yval,y_pred))

        cm = confusion_matrix(yval,y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval,y_pred)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

        print(classification_report(yval,y_pred))

        fpr, tpr, _ = roc_curve(yval,y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)

        print("Precision:   ",TP/(TP+FP))
        
        acc.append(accuracy_score(yval, y_pred))
        
        cvscores.append(mcc)
        
        sn.append(Sensitivity)
        sp.append(Specificity)

        auc_scores.append(roc_auc_test) 


    print("Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("\n\n\n\n\n\n")

Seed :  848295
(9770, 1024)
(9770,)


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.6336088044386692
Confusion Matrix : 
 [[401  87]
 [ 92 397]]
Accuracy on test set:    0.8167860798362334
Sensitivity:    0.8118609406952966 	 Specificity:    0.8217213114754098
              precision    recall  f1-score   support

           0       0.81      0.82      0.82       488
           1       0.82      0.81      0.82       489

    accuracy                           0.82       977
   macro avg       0.82      0.82      0.82       977
weighted avg       0.82      0.82      0.82       977

Area Under Curve:    0.8167911260853532
Precision:    0.8202479338842975


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.6417974878028009
Confusion Matrix : 
 [[403  85]
 [ 90 399]]
Accuracy on test set:    0.8208802456499488
Sensitivity:    0.8159509202453987 	 Specificity:    0.8258196721311475
              precision    recall  f1-score   support

           0       0.82      0.83      0.82       488
           1       0.82      0.82      0.82       489

    accuracy                           0.82       977
   macro avg       0.82      0.82      0.82       977
weighted avg       0.82      0.82      0.82       977

Area Under Curve:    0.8208852961882731
Precision:    0.8243801652892562


...

/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.658523527523186
Confusion Matrix : 
 [[414  75]
 [ 92 396]]
Accuracy on test set:    0.8290685772773797
Sensitivity:    0.8114754098360656 	 Specificity:    0.8466257668711656
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       489
           1       0.84      0.81      0.83       488

    accuracy                           0.83       977
   macro avg       0.83      0.83      0.83       977
weighted avg       0.83      0.83      0.83       977

Area Under Curve:    0.8290505883536157
Precision:    0.8407643312101911


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.6317060625913142
Confusion Matrix : 
 [[405  84]
 [ 96 392]]
Accuracy on test set:    0.8157625383828045
Sensitivity:    0.8032786885245902 	 Specificity:    0.8282208588957055
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       489
           1       0.82      0.80      0.81       488

    accuracy                           0.82       977
   macro avg       0.82      0.82      0.82       977
weighted avg       0.82      0.82      0.82       977

Area Under Curve:    0.8157497737101479
Precision:    0.8235294117647058
Matthews Correlation :  0.6479082768012435
Confusion Matrix : 
 [[402  87]
 [ 85 403]]
Accuracy on test set:    0.8239508700102354
Sensitivity:    0.8258196721311475 	 Specificity:    0.8220858895705522
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       489
           1       0.82      0.83      0.82       488

    accuracy                  

/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# Independent Testing

In [9]:
os.chdir("/home/t326h379/OGP")


df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

a = random.sample(range(1, 1000000), 3)

In [10]:
# Independent Test Dataset
df_negative_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Negative_11466_Sites_less.txt',header=None)

df_positive_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Positive_375_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive_test.columns = Header_name

df_negative_test.columns = Header_name


frames_test = [df_positive_test, df_negative_test]

O_linked_testing = pd.concat(frames_test,ignore_index = True)

df_Test_array = O_linked_testing.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Test_array = np.array(df_Test_array)

X_test_full = df_Test_array

y_test_full = np.array([1]*374+[0]*11466)

In [13]:
print(X_train_full.shape,y_train_full.shape,X_test_full.shape,y_test_full.shape)

(119029, 1024) (119029,) (11840, 1024) (11840,)


In [14]:
for i in a:
    seed = i
    print("Seed : ",seed)
    rus = RandomUnderSampler(random_state = seed)
    X_train, y_train = rus.fit_resample(X_train_full,y_train_full)

    X_train, y_train = shuffle(X_train,y_train)

    rus = RandomUnderSampler(random_state = seed)

    clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

    clf.fit(X_train, y_train)
    
    X_test_random_undersample, y_test_random_undersample = rus.fit_resample(X_test_full,y_test_full)


    y_pred = np.array(clf.predict(X_test_random_undersample))

    confusion = confusion_matrix(y_test_random_undersample,y_pred)

    print("Matthews Correlation : ",matthews_corrcoef(y_test_random_undersample,y_pred))
    print("Confusion Matrix : \n",confusion_matrix(y_test_random_undersample,y_pred))
    print("Accuracy on test set:   ",accuracy_score(y_test_random_undersample,y_pred))

    cm = confusion_matrix(y_test_random_undersample,y_pred)

    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]

    mcc = matthews_corrcoef(y_test_random_undersample,y_pred)

    Sensitivity = TP/(TP+FN)

    Specificity = TN/(TN+FP)

    print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

    print(classification_report(y_test_random_undersample,y_pred))

    fpr, tpr, _ = roc_curve(y_test_random_undersample,y_pred)

    roc_auc_test = auc(fpr,tpr)



    print("Area Under Curve:   ",roc_auc_test)

    print("Precision:   ",TP/(TP+FP))

Seed :  437456


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.5329641044922544
Confusion Matrix : 
 [[314  60]
 [117 257]]
Accuracy on test set:    0.7633689839572193
Sensitivity:    0.6871657754010695 	 Specificity:    0.839572192513369
              precision    recall  f1-score   support

           0       0.73      0.84      0.78       374
           1       0.81      0.69      0.74       374

    accuracy                           0.76       748
   macro avg       0.77      0.76      0.76       748
weighted avg       0.77      0.76      0.76       748

Area Under Curve:    0.7633689839572192
Precision:    0.8107255520504731
Seed :  664482


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.542912203416473
Confusion Matrix : 
 [[314  60]
 [113 261]]
Accuracy on test set:    0.7687165775401069
Sensitivity:    0.6978609625668449 	 Specificity:    0.839572192513369
              precision    recall  f1-score   support

           0       0.74      0.84      0.78       374
           1       0.81      0.70      0.75       374

    accuracy                           0.77       748
   macro avg       0.77      0.77      0.77       748
weighted avg       0.77      0.77      0.77       748

Area Under Curve:    0.7687165775401069
Precision:    0.8130841121495327
Seed :  111916


...

/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.546404180620424
Confusion Matrix : 
 [[310  64]
 [107 267]]
Accuracy on test set:    0.7713903743315508
Sensitivity:    0.713903743315508 	 Specificity:    0.8288770053475936
              precision    recall  f1-score   support

           0       0.74      0.83      0.78       374
           1       0.81      0.71      0.76       374

    accuracy                           0.77       748
   macro avg       0.78      0.77      0.77       748
weighted avg       0.78      0.77      0.77       748

Area Under Curve:    0.7713903743315508
Precision:    0.8066465256797583
Seed :  393435


/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Matthews Correlation :  0.5475000633157839
Confusion Matrix : 
 [[313  61]
 [110 264]]
Accuracy on test set:    0.7713903743315508
Sensitivity:    0.7058823529411765 	 Specificity:    0.8368983957219251
              precision    recall  f1-score   support

           0       0.74      0.84      0.79       374
           1       0.81      0.71      0.76       374

    accuracy                           0.77       748
   macro avg       0.78      0.77      0.77       748
weighted avg       0.78      0.77      0.77       748

Area Under Curve:    0.7713903743315509
Precision:    0.8123076923076923
Seed :  747864
Matthews Correlation :  0.4900340716007037
Confusion Matrix : 
 [[300  74]
 [118 256]]
Accuracy on test set:    0.7433155080213903
Sensitivity:    0.6844919786096256 	 Specificity:    0.8021390374331551
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       374
           1       0.78      0.68      0.73       374

    accuracy   

/home/t326h379/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# Thank You!!!